## Production-Level Explanation & Insights: modal_mlflow_sagemaker.ipynb (Excerpted Sections)

---

### **1. Data Extraction from AWS Redshift**

#### **What the Code Does**
- **Connects to AWS Redshift** using the Data API (`boto3.client('redshift-data')`).
- **Executes a SQL query** to fetch customer data (`SELECT * FROM public.customer LIMIT 10;`).
- **Polls for query completion** using a loop (`describe_statement`).
- **Retrieves results** and parses them into a Python list.
- **Extracts column names** and values, handling different Redshift data types.
- **Builds a Pandas DataFrame** from the results.
- **Saves the DataFrame as a CSV** (`customer_data.csv`).

#### **Production Insights**
- **Why Redshift Data API?**  
  Enables serverless, secure, and scalable access to Redshift without JDBC/ODBC drivers.
- **Polling for Completion:**  
  Ensures the query is finished before fetching results, avoiding race conditions.
- **Type Extraction:**  
  Handles multiple Redshift types (`stringValue`, `longValue`, etc.) for robust parsing.
- **CSV Output:**  
  Standardizes data for downstream ML and auditability.

#### **Best Practices**
- **Error Handling:**  
  Raises exceptions on query failure for reliability.
- **Security:**  
  Credentials should be managed via IAM roles, not hardcoded.
- **Scalability:**  
  For large datasets, consider pagination or chunked downloads.

---

### **2. Data Loading & Preprocessing Pipeline**

#### **What the Code Does**
- **Loads CSV data** into Pandas.
- **Initial Exploration:**  
  Prints shape, columns, and sample rows for verification.
- **Logging Setup:**  
  Configures logging to file and console for traceability.
- **Automatic Column Type Detection:**  
  Identifies numerical, categorical, binary, ID, and target columns using heuristics.
- **Missing Value Analysis:**  
  Reports missing counts and percentages.
- **Cleaning:**  
  Drops ID columns, fills missing values (median for numerical, mode for categorical/binary).
- **Binary Encoding:**  
  Converts binary columns to 0/1 using intelligent mapping.
- **Preprocessing Pipeline:**  
  Builds a `ColumnTransformer` for scaling numerical and one-hot encoding categorical features.
- **Transformation:**  
  Applies pipeline, adds encoded binary and target columns.
- **Saves Results:**  
  Stores processed data, pipeline, and metadata for reproducibility.

#### **Production Insights**
- **Why Automated Detection?**  
  Reduces manual errors and adapts to schema changes.
- **Why Logging?**  
  Essential for debugging, compliance, and audit trails.
- **Why Separate Handling for Binary/Categorical/Numerical?**  
  Ensures correct encoding and scaling, which is critical for ML algorithms.
- **Why Save Artifacts?**  
  Guarantees reproducibility and enables consistent inference in production.

#### **Best Practices**
- **Limit One-Hot Encoding:**  
  Avoids memory issues by skipping columns with too many categories.
- **Metadata Storage:**  
  Facilitates future audits and model explainability.
- **Pipeline Versioning:**  
  Timestamped outputs for traceability.

---

### **3. ML Pipeline Class (`SalesConversionMLPipeline`)**

#### **What the Code Does**
- **Initializes ML Pipeline:**  
  Loads config, sets up AWS/SageMaker/MLflow clients, and stores data.
- **Data Preparation:**  
  Splits data into train/test, scales features, and checks for target column.
- **Model Definition:**  
  Specifies four algorithms (Random Forest, Gradient Boosting, Logistic Regression, SVM) with hyperparameter grids.
- **Training & Tuning:**  
  Performs grid search with cross-validation, logs all runs to MLflow.
- **Evaluation:**  
  Calculates metrics (accuracy, precision, recall, F1, ROC-AUC), logs confusion matrix, classification report, and feature importance.
- **Best Model Selection:**  
  Picks the model with highest ROC-AUC, logs selection and comparison table.
- **Artifact Preparation:**  
  Bundles model, scaler, feature names, parameters, and metrics; uploads to S3.
- **Inference Code Generation:**  
  Creates and uploads a Python script for production inference.
- **Model Registration:**  
  Registers only the best model in MLflow Model Registry (SageMaker registration disabled due to ECR issues).
- **Reporting:**  
  Generates comprehensive report, experiment info, and model comparison DataFrame.

#### **Production Insights**
- **Why MLflow?**  
  Centralizes experiment tracking, model comparison, and registry for reproducibility and audit.
- **Why Only Best Model Registration?**  
  Reduces clutter, focuses on deployable models, and simplifies governance.
- **Why S3 for Artifacts?**  
  Ensures scalable, secure, and persistent storage for production deployment.
- **Why Custom Inference Code?**  
  Guarantees consistency between training and production inference.

#### **Best Practices**
- **Stratified Splits:**  
  Maintains class balance for fair evaluation.
- **Comprehensive Metrics:**  
  Multiple metrics provide a holistic view of model performance.
- **Versioning & Logging:**  
  All outputs are timestamped and logged for traceability.
- **Error Handling:**  
  Exceptions are caught and logged for reliability.

---

### **Summary Table**

| Step                | Purpose                                   | Production Reasoning                |
|---------------------|-------------------------------------------|-------------------------------------|
| Data Extraction     | Securely fetch data from Redshift         | Serverless, scalable, auditable     |
| Preprocessing       | Clean, encode, and transform data         | Automation, consistency, reproducibility |
| Model Training      | Find best algorithm and parameters        | Experimentation, optimization       |
| Evaluation          | Assess model performance                  | Holistic metrics, explainability    |
| Best Model Selection| Choose deployable model                   | Transparent, metric-driven          |
| Artifact Prep       | Bundle all inference components           | Reproducibility, deployment readiness|
| Inference Code      | Enable production inference               | Consistency, scalability            |
| Model Registration  | Lifecycle management                      | Versioning, audit, deployment       |
| Reporting           | Communicate results                       | Business decision support           |

---

## **Key Takeaways for Production**

- **Automation** and **logging** are critical for reliability and auditability.
- **Only the best model** is registered for deployment, reducing operational complexity.
- **All artifacts** (data, pipeline, model, code) are versioned and stored for reproducibility.
- **MLflow integration** enables experiment tracking, comparison, and registry management.
- **SageMaker Model Registry** is disabled due to ECR issues; MLflow is used for model lifecycle.
- **Comprehensive reporting** supports business and technical stakeholders.

---

**This notebook is designed for robust, scalable, and reproducible machine learning in production environments.**  
If you need code-level comments or further breakdown of any section, let me know!

In [27]:
import boto3
import time
import pandas as pd

# Create Redshift Data API client
client = boto3.client('redshift-data', region_name='ap-south-1')

# Execute the SQL query
response = client.execute_statement(
    WorkgroupName='aravind-workgroup',
    Database='dev',
    Sql='SELECT * FROM public.customer LIMIT 10;'
)

statement_id = response['Id']

# Wait until the query is finished
while True:
    status = client.describe_statement(Id=statement_id)
    if status['Status'] in ['FAILED', 'ABORTED']:
        raise Exception(f"Query failed: {status.get('Error', 'Unknown error')}")
    elif status['Status'] == 'FINISHED':
        break
    time.sleep(1)

# Get the query results
results = client.get_statement_result(Id=statement_id)

# Extract column names
column_names = [col['name'] for col in results['ColumnMetadata']]

# Function to extract correct value type
def extract_value(col):
    if 'stringValue' in col:
        return col['stringValue']
    elif 'longValue' in col:
        return col['longValue']
    elif 'booleanValue' in col:
        return col['booleanValue']
    elif 'doubleValue' in col:
        return col['doubleValue']
    else:
        return None  # or ""

# Extract rows
data = []
for record in results['Records']:
    row = [extract_value(col) for col in record]
    data.append(row)

# Create DataFrame
df = pd.DataFrame(data, columns=column_names)

# Save as CSV
df.to_csv('customer_data.csv', index=False)
print("Data saved to customer_data.csv")


Data saved to customer_data.csv


In [28]:
df = pd.read_csv('customer_data.csv')
df

prospect id  lead number              lead origin  \
0  8b94b8c4-7107-4c12-8a66-531d0ed974c4       615582  Landing Page Submission   
1  a4c0a4e4-c5d8-4e48-8a4f-030611dde534       588939  Landing Page Submission   
2  889c23c8-84d0-4936-97ad-7b438dc2e2d7       621242  Landing Page Submission   
3  85dfdbea-fb6c-4428-a59c-59155bedea4a       589803  Landing Page Submission   
4  cbc292d8-752f-47ea-92c6-07d4247638d5       651441  Landing Page Submission   
5  4bae149e-3b30-4262-8f78-50685d52125b       650484                      API   
6  352e5985-f3f2-4188-943c-332ca95e8d09       657038                      API   
7  ce68ad09-cc22-4a59-bf1f-dafdb572d1af       649329  Landing Page Submission   
8  18a900d2-86e2-4e4a-b475-70ee71a2867e       587559  Landing Page Submission   
9  44efc6d4-fdfd-4638-bf55-38da6818f911       643897            Lead Add Form   

      lead source do not email do not call  converted  totalvisits  \
0          Google           No          No          1          8.0   
1  Direct Traffic          Yes          No          0          2.0   
2  Direct Traffic          Yes          No          0          1.0   
3  Direct Traffic           No          No          0          2.0   
4          Google           No          No          0          3.0   
5          Google           No          No          0          3.0   
6  Referral Sites           No          No          1          5.0   
7  Direct Traffic           No          No          0          7.0   
8  Direct Traffic           No          No          0          2.0   
9       Reference           No          No          1          0.0   

   total time spent on website  page views per visit  ...  \
0                          252                  2.67  ...   
1                          929                  2.00  ...   
2                            2                  1.00  ...   
3                          323                  2.00  ...   
4                          201                  3.00  ...   
5                          388                  1.50  ...   
6                         1536                  5.00  ...   
7                          219                  2.33  ...   
8                           24                  2.00  ...   
9                            0                  0.00  ...   

  get updates on dm content    lead profile               city  \
0                        No          Select             Mumbai   
1                        No             NaN       Other Cities   
2                        No             NaN       Other Cities   
3                        No          Select             Mumbai   
4                        No             NaN             Mumbai   
5                        No          Select             Select   
6                        No  Potential Lead  Thane & Outskirts   
7                        No          Select       Other Cities   
8                        No          Select             Mumbai   
9                        No  Potential Lead             Select   

  asymmetrique activity index asymmetrique profile index  \
0                         NaN                        NaN   
1                   02.Medium                  02.Medium   
2                         NaN                        NaN   
3                   02.Medium                    01.High   
4                   02.Medium                    01.High   
5                     01.High                  02.Medium   
6                   02.Medium                  02.Medium   
7                   02.Medium                  02.Medium   
8                   02.Medium                    01.High   
9                   02.Medium                    01.High   

  asymmetrique activity score asymmetrique profile score  \
0                         NaN                        NaN   
1                        14.0                       16.0   
2                         NaN                        NaN   
3                        14.0                       18.0   
4    

In [25]:
# Universal Data Preprocessing Pipeline
# This version automatically detects column types and adapts to your dataset

import pandas as pd
import numpy as np
import os
import pickle
import logging
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

# Configuration - UPDATE THESE PATHS
INPUT_PATH = "customer_data.csv"
OUTPUT_PATH = "pre_processed"

print(f"Input file: {INPUT_PATH}")
print(f"Output folder: {OUTPUT_PATH}")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_PATH, exist_ok=True)
print("✅ Output directory ready!")

def setup_logging(output_path):
    """Set up logging configuration"""
    log_dir = os.path.join(output_path, 'logs')
    os.makedirs(log_dir, exist_ok=True)
    
    log_file = os.path.join(log_dir, f'preprocessing_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
    
    # Clear any existing handlers
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

# Initialize logger
logger = setup_logging(OUTPUT_PATH)
logger.info("📊 Starting Universal Data Preprocessing")

def load_data(input_path):
    """Load data from CSV file"""
    logger.info(f"Loading data from: {input_path}")
    try:
        df = pd.read_csv(input_path)
        logger.info(f"Data loaded successfully. Shape: {df.shape}")
        return df
    except Exception as e:
        logger.error(f"Error loading data: {str(e)}")
        raise

# Load the data
df = load_data(INPUT_PATH)

# Display basic information
print(f"📈 Dataset Shape: {df.shape}")
print(f"📊 Columns: {len(df.columns)}")
print("\n🔍 First few rows:")
print(df.head())

print(f"\n📋 All Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n📋 Column Info:")
print(df.info())

def detect_column_types(df):
    """Automatically detect column types based on data patterns"""
    numerical_cols = []
    categorical_cols = []
    binary_cols = []
    id_cols = []
    target_cols = []
    
    for col in df.columns:
        # Skip if mostly null
        if df[col].isnull().sum() / len(df) > 0.95:
            logger.info(f"Skipping {col} - mostly null values")
            continue
            
        # Check for ID columns (unique values > 80% of rows)
        unique_ratio = df[col].nunique() / len(df)
        if unique_ratio > 0.8:
            id_cols.append(col)
            logger.info(f"Detected ID column: {col}")
            continue
            
        # Check for target columns (common target names)
        target_keywords = ['converted', 'target', 'label', 'class', 'outcome', 'response', 'churn', 'buy', 'purchase']
        if any(keyword in col.lower() for keyword in target_keywords):
            target_cols.append(col)
            logger.info(f"Detected potential target column: {col}")
            continue
            
        # Check data type
        if df[col].dtype in ['int64', 'float64']:
            # Check if it's actually categorical (few unique values)
            if df[col].nunique() <= 10:
                categorical_cols.append(col)
                logger.info(f"Detected categorical (numeric): {col}")
            else:
                numerical_cols.append(col)
                logger.info(f"Detected numerical: {col}")
        else:
            # String/object type
            unique_vals = df[col].nunique()
            
            # Check for binary columns
            if unique_vals <= 3:  # Including null
                non_null_vals = df[col].dropna().unique()
                if len(non_null_vals) == 2:
                    # Check if binary patterns
                    val_set = set([str(v).lower() for v in non_null_vals])
                    binary_patterns = [
                        {'yes', 'no'}, {'y', 'n'}, {'true', 'false'}, {'1', '0'},
                        {'male', 'female'}, {'m', 'f'}, {'active', 'inactive'}
                    ]
                    if any(val_set == pattern for pattern in binary_patterns):
                        binary_cols.append(col)
                        logger.info(f"Detected binary column: {col}")
                        continue
            
            categorical_cols.append(col)
            logger.info(f"Detected categorical (string): {col}")
    
    return numerical_cols, categorical_cols, binary_cols, id_cols, target_cols

# Detect column types
numerical_cols, categorical_cols, binary_cols, id_cols, target_cols = detect_column_types(df)

print(f"\n🔍 COLUMN TYPE DETECTION RESULTS:")
print(f"  🔢 Numerical columns ({len(numerical_cols)}): {numerical_cols}")
print(f"  🏷️  Categorical columns ({len(categorical_cols)}): {categorical_cols}")
print(f"  🔘 Binary columns ({len(binary_cols)}): {binary_cols}")
print(f"  🆔 ID columns ({len(id_cols)}): {id_cols}")
print(f"  🎯 Target columns ({len(target_cols)}): {target_cols}")

def analyze_missing_values(df):
    """Analyze missing values in the dataset"""
    print("\n🔍 Missing Values Analysis:")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing_data,
        'Missing_Percentage': missing_percent
    }).sort_values('Missing_Count', ascending=False)
    
    missing_cols = missing_df[missing_df['Missing_Count'] > 0]
    if len(missing_cols) > 0:
        print(missing_cols)
    else:
        print("✅ No missing values found!")
    
    return missing_cols

# Analyze missing values
missing_analysis = analyze_missing_values(df)

def clean_dataset(df, id_cols):
    """Remove ID columns and handle basic cleaning"""
    df_cleaned = df.copy()
    
    # Drop ID columns
    if id_cols:
        df_cleaned = df_cleaned.drop(columns=id_cols)
        logger.info(f"Dropped ID columns: {id_cols}")
        print(f"✅ Dropped {len(id_cols)} ID columns: {id_cols}")
    
    return df_cleaned

# Clean dataset
df_cleaned = clean_dataset(df, id_cols)

def handle_missing_values(df, numerical_cols, categorical_cols, binary_cols):
    """Handle missing values with appropriate strategies"""
    df_filled = df.copy()
    
    print("\n🔧 Handling Missing Values:")
    
    # Handle numerical columns
    if numerical_cols:
        print("  🔢 Numerical columns:")
        for col in numerical_cols:
            if col in df_filled.columns and df_filled[col].isnull().sum() > 0:
                before_missing = df_filled[col].isnull().sum()
                median_val = df_filled[col].median()
                df_filled[col].fillna(median_val, inplace=True)
                print(f"    • {col}: {before_missing} missing → filled with median {median_val:.2f}")
                logger.info(f"Filled {col} with median: {median_val}")
    
    # Handle categorical columns
    if categorical_cols:
        print("  🏷️  Categorical columns:")
        for col in categorical_cols:
            if col in df_filled.columns and df_filled[col].isnull().sum() > 0:
                before_missing = df_filled[col].isnull().sum()
                mode_val = df_filled[col].mode()
                mode_value = mode_val[0] if len(mode_val) > 0 else 'Unknown'
                df_filled[col].fillna(mode_value, inplace=True)
                print(f"    • {col}: {before_missing} missing → filled with mode '{mode_value}'")
                logger.info(f"Filled {col} with mode: {mode_value}")
    
    # Handle binary columns
    if binary_cols:
        print("  🔘 Binary columns:")
        for col in binary_cols:
            if col in df_filled.columns and df_filled[col].isnull().sum() > 0:
                before_missing = df_filled[col].isnull().sum()
                # Fill with the most common value or 'No'/'False'
                mode_val = df_filled[col].mode()
                if len(mode_val) > 0:
                    fill_value = mode_val[0]
                else:
                    # Default to negative response
                    unique_vals = df_filled[col].dropna().unique()
                    negative_vals = ['no', 'n', 'false', '0', 'inactive']
                    fill_value = next((val for val in unique_vals 
                                     if str(val).lower() in negative_vals), unique_vals[0])
                
                df_filled[col].fillna(fill_value, inplace=True)
                print(f"    • {col}: {before_missing} missing → filled with '{fill_value}'")
                logger.info(f"Filled {col} with: {fill_value}")
    
    return df_filled

# Handle missing values
df_filled = handle_missing_values(df_cleaned, numerical_cols, categorical_cols, binary_cols)

# Update column lists after cleaning (remove dropped columns)
numerical_cols = [col for col in numerical_cols if col in df_filled.columns]
categorical_cols = [col for col in categorical_cols if col in df_filled.columns]
binary_cols = [col for col in binary_cols if col in df_filled.columns]
target_cols = [col for col in target_cols if col in df_filled.columns]

print(f"\n✅ Missing values handled. Remaining missing values: {df_filled.isnull().sum().sum()}")

def encode_binary_columns(df, binary_cols):
    """Encode binary columns to 0/1"""
    df_encoded = df.copy()
    
    if binary_cols:
        print("\n🔄 Encoding Binary Columns:")
        for col in binary_cols:
            print(f"  • {col}:")
            
            # Show original values
            value_counts = df_encoded[col].value_counts()
            print(f"    Original values: {dict(value_counts)}")
            
            # Create mapping (first unique value = 0, second = 1)
            unique_vals = df_encoded[col].unique()
            unique_vals = [val for val in unique_vals if pd.notna(val)]
            
            if len(unique_vals) >= 2:
                # Try to map intelligently
                val_lower = [str(val).lower() for val in unique_vals]
                positive_vals = ['yes', 'y', 'true', '1', 'active', 'male', 'm']
                
                # Find positive value
                positive_val = None
                for val, val_low in zip(unique_vals, val_lower):
                    if val_low in positive_vals:
                        positive_val = val
                        break
                
                if positive_val is None:
                    positive_val = unique_vals[1]  # Default to second value
                
                # Create binary encoding
                df_encoded[f"{col}_encoded"] = (df_encoded[col] == positive_val).astype(int)
                
                print(f"    Encoded: '{positive_val}' → 1, others → 0")
                print(f"    Result: {dict(df_encoded[f'{col}_encoded'].value_counts())}")
                
                logger.info(f"Encoded binary column {col}: '{positive_val}' → 1")
    
    return df_encoded

# Encode binary columns
df_with_binary = encode_binary_columns(df_filled, binary_cols)

def create_preprocessing_pipeline(numerical_cols, categorical_cols):
    """Create sklearn preprocessing pipeline"""
    transformers = []
    
    # Add numerical transformer if we have numerical columns
    if numerical_cols:
        transformers.append(('num', MinMaxScaler(), numerical_cols))
        logger.info(f"Added numerical transformer for: {numerical_cols}")
        print(f"🔢 Will scale numerical features: {numerical_cols}")
    
    # Add categorical transformer if we have categorical columns
    if categorical_cols:
        # Limit categories to prevent memory issues
        limited_categorical = []
        for col in categorical_cols:
            unique_count = df_with_binary[col].nunique()
            if unique_count <= 50:  # Reasonable limit for one-hot encoding
                limited_categorical.append(col)
            else:
                logger.warning(f"Skipping {col} - too many categories ({unique_count})")
                print(f"⚠️  Skipping {col} - too many categories ({unique_count})")
        
        if limited_categorical:
            transformers.append(('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), limited_categorical))
            logger.info(f"Added categorical transformer for: {limited_categorical}")
            print(f"🏷️  Will one-hot encode categorical features: {limited_categorical}")
    
    if not transformers:
        logger.warning("No transformers created - no suitable columns found")
        return None, [], []
    
    # Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='drop'
    )
    
    return preprocessor, numerical_cols, limited_categorical if 'limited_categorical' in locals() else categorical_cols

# Create preprocessing pipeline
preprocessor, final_numerical_cols, final_categorical_cols = create_preprocessing_pipeline(numerical_cols, categorical_cols)

def apply_preprocessing(df_with_binary, preprocessor, final_numerical_cols, final_categorical_cols, binary_cols, target_cols):
    """Apply the preprocessing pipeline"""
    
    if preprocessor is None:
        logger.warning("No preprocessing pipeline available")
        df_final = pd.DataFrame(index=df_with_binary.index)
    else:
        # Prepare features for pipeline
        feature_cols = final_numerical_cols + final_categorical_cols
        
        if feature_cols:
            X = df_with_binary[feature_cols]
            
            print(f"🔄 Applying preprocessing to {len(feature_cols)} features...")
            
            # Fit and transform
            X_transformed = preprocessor.fit_transform(X)
            
            # Get feature names
            feature_names = []
            
            # Add numerical feature names
            if final_numerical_cols:
                feature_names.extend(final_numerical_cols)
            
            # Add categorical feature names
            if final_categorical_cols:
                try:
                    cat_names = preprocessor.named_transformers_['cat'].get_feature_names_out(final_categorical_cols)
                    feature_names.extend(cat_names)
                except:
                    # Fallback
                    n_cat_features = X_transformed.shape[1] - len(final_numerical_cols)
                    cat_names = [f"cat_feature_{i}" for i in range(n_cat_features)]
                    feature_names.extend(cat_names)
            
            # Create final dataframe
            df_final = pd.DataFrame(X_transformed, columns=feature_names, index=df_with_binary.index)
            
            print(f"✅ Created {len(feature_names)} transformed features")
            logger.info(f"Preprocessing complete. Output shape: {df_final.shape}")
        else:
            df_final = pd.DataFrame(index=df_with_binary.index)
            logger.warning("No features to transform")
    
    # Add binary encoded features
    binary_encoded_cols = [f"{col}_encoded" for col in binary_cols if f"{col}_encoded" in df_with_binary.columns]
    for col in binary_encoded_cols:
        df_final[col] = df_with_binary[col]
    
    if binary_encoded_cols:
        print(f"✅ Added {len(binary_encoded_cols)} binary encoded features")
    
    # Add target columns
    for target_col in target_cols:
        if target_col in df_with_binary.columns:
            df_final[target_col] = df_with_binary[target_col]
            print(f"✅ Added target column: {target_col}")
            logger.info(f"Added target column: {target_col}")
    
    return df_final

# Apply preprocessing
df_final = apply_preprocessing(df_with_binary, preprocessor, final_numerical_cols, final_categorical_cols, binary_cols, target_cols)

print(f"\n🎉 PREPROCESSING COMPLETE!")
print(f"📊 Final dataset shape: {df_final.shape}")

if not df_final.empty:
    print(f"📋 Final columns ({len(df_final.columns)}):")
    for i, col in enumerate(df_final.columns, 1):
        print(f"  {i:2d}. {col}")
    
    # Show target distribution if available
    for target_col in target_cols:
        if target_col in df_final.columns:
            print(f"\n🎯 Target Distribution ({target_col}):")
            print(df_final[target_col].value_counts())
    
    print(f"\n📊 Sample of Final Data:")
    print(df_final.head())
    
    # Only call describe if we have columns
    if len(df_final.columns) > 0:
        print(f"\n📈 Final Data Statistics:")
        print(df_final.describe())
else:
    print("⚠️  Warning: Final dataset is empty. Please check your data and column detection.")

def save_results(df_final, preprocessor, output_path, final_numerical_cols, final_categorical_cols, binary_cols, target_cols):
    """Save all results"""
    if df_final.empty:
        print("❌ Cannot save empty dataset")
        return None, None, None
    
    logger.info(f"Saving results to: {output_path}")
    
    # Create directories
    processed_dir = os.path.join(output_path, 'processed')
    os.makedirs(processed_dir, exist_ok=True)
    
    # Save processed data
    processed_data_path = os.path.join(processed_dir, 'preprocessed_data.csv')
    df_final.to_csv(processed_data_path, index=False)
    print(f"💾 Saved processed data: {processed_data_path}")
    
    # Save pipeline if available
    pipeline_path = None
    if preprocessor is not None:
        pipeline_path = os.path.join(processed_dir, 'preprocessing_pipeline.pkl')
        with open(pipeline_path, 'wb') as f:
            pickle.dump(preprocessor, f)
        print(f"💾 Saved preprocessing pipeline: {pipeline_path}")
    
    # Save metadata
    metadata = {
        'original_shape': df.shape,
        'final_shape': df_final.shape,
        'numerical_features': final_numerical_cols,
        'categorical_features': final_categorical_cols,
        'binary_features': binary_cols,
        'target_features': target_cols,
        'processing_timestamp': datetime.now().isoformat(),
        'total_features': len(df_final.columns) - len(target_cols)
    }
    
    metadata_path = os.path.join(processed_dir, 'metadata.pkl')
    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)
    print(f"💾 Saved metadata: {metadata_path}")
    
    return processed_data_path, pipeline_path, metadata_path

# Save results
if not df_final.empty:
    paths = save_results(df_final, preprocessor, OUTPUT_PATH, final_numerical_cols, final_categorical_cols, binary_cols, target_cols)
    
    print(f"\n🚀 SUCCESS! Your data is now ready for machine learning.")
    print(f"📁 Check the '{OUTPUT_PATH}/processed/' folder for all outputs.")
else:
    print(f"\n❌ Preprocessing failed - no suitable columns found in your dataset.")
    print(f"🔍 Please check that your CSV file has the expected structure.")

2025-07-19 17:33:03,209 - INFO - 📊 Starting Universal Data Preprocessing
2025-07-19 17:33:03,210 - INFO - Loading data from: customer_data.csv
2025-07-19 17:33:03,214 - INFO - Data loaded successfully. Shape: (10, 37)
2025-07-19 17:33:03,230 - INFO - Detected ID column: prospect id
2025-07-19 17:33:03,232 - INFO - Detected ID column: lead number
2025-07-19 17:33:03,233 - INFO - Detected categorical (string): lead origin
2025-07-19 17:33:03,235 - INFO - Detected categorical (string): lead source
2025-07-19 17:33:03,237 - INFO - Detected binary column: do not email
2025-07-19 17:33:03,239 - INFO - Detected categorical (string): do not call
2025-07-19 17:33:03,240 - INFO - Detected potential target column: converted
2025-07-19 17:33:03,242 - INFO - Detected categorical (numeric): totalvisits
2025-07-19 17:33:03,243 - INFO - Detected ID column: total time spent on website
2025-07-19 17:33:03,244 - INFO - Detected categorical (numeric): page views per visit
2025-07-19 17:33:03,246 - INFO - 

✅ All libraries imported successfully!
Input file: customer_data.csv
Output folder: pre_processed
✅ Output directory ready!
📈 Dataset Shape: (10, 37)
📊 Columns: 37

🔍 First few rows:
                            prospect id  lead number              lead origin  \
0  8b94b8c4-7107-4c12-8a66-531d0ed974c4       615582  Landing Page Submission   
1  a4c0a4e4-c5d8-4e48-8a4f-030611dde534       588939  Landing Page Submission   
2  889c23c8-84d0-4936-97ad-7b438dc2e2d7       621242  Landing Page Submission   
3  85dfdbea-fb6c-4428-a59c-59155bedea4a       589803  Landing Page Submission   
4  cbc292d8-752f-47ea-92c6-07d4247638d5       651441  Landing Page Submission   

      lead source do not email do not call  converted  totalvisits  \
0          Google           No          No          1          8.0   
1  Direct Traffic          Yes          No          0          2.0   
2  Direct Traffic          Yes          No          0          1.0   
3  Direct Traffic           No          No        

2025-07-19 17:33:03,344 - INFO - Added categorical transformer for: ['lead origin', 'lead source', 'do not call', 'totalvisits', 'page views per visit', 'last activity', 'country', 'specialization', 'how did you hear about x education', 'what is your current occupation', 'what matters most to you in choosing a course', 'search', 'magazine', 'newspaper article', 'x education forums', 'newspaper', 'digital advertisement', 'through recommendations', 'receive more updates about our courses', 'tags', 'lead quality', 'update me on supply chain content', 'get updates on dm content', 'lead profile', 'city', 'asymmetrique activity index', 'asymmetrique profile index', 'asymmetrique activity score', 'asymmetrique profile score', 'i agree to pay the amount through cheque', 'last notable activity']
2025-07-19 17:33:03,361 - INFO - Preprocessing complete. Output shape: (10, 51)
2025-07-19 17:33:03,363 - INFO - Added target column: converted


🏷️  Will one-hot encode categorical features: ['lead origin', 'lead source', 'do not call', 'totalvisits', 'page views per visit', 'last activity', 'country', 'specialization', 'how did you hear about x education', 'what is your current occupation', 'what matters most to you in choosing a course', 'search', 'magazine', 'newspaper article', 'x education forums', 'newspaper', 'digital advertisement', 'through recommendations', 'receive more updates about our courses', 'tags', 'lead quality', 'update me on supply chain content', 'get updates on dm content', 'lead profile', 'city', 'asymmetrique activity index', 'asymmetrique profile index', 'asymmetrique activity score', 'asymmetrique profile score', 'i agree to pay the amount through cheque', 'last notable activity']
🔄 Applying preprocessing to 31 features...
✅ Created 51 transformed features
✅ Added 2 binary encoded features
✅ Added target column: converted

🎉 PREPROCESSING COMPLETE!
📊 Final dataset shape: (10, 54)
📋 Final columns (54):

2025-07-19 17:33:03,458 - INFO - Saving results to: pre_processed


       lead origin_Landing Page Submission  lead origin_Lead Add Form  \
count                            10.000000                  10.000000   
mean                              0.700000                   0.100000   
std                               0.483046                   0.316228   
min                               0.000000                   0.000000   
25%                               0.250000                   0.000000   
50%                               1.000000                   0.000000   
75%                               1.000000                   0.000000   
max                               1.000000                   1.000000   

       lead source_Google  lead source_Reference  lead source_Referral Sites  \
count           10.000000              10.000000                   10.000000   
mean             0.300000               0.100000                    0.100000   
std              0.483046               0.316228                    0.316228   
min              0.000

In [26]:
"""
Sales Conversion Prediction MLOps Pipeline - Fixed Version with MLflow
====================================================================

This is a complete MLOps pipeline for sales conversion prediction that includes:
- Data loading from preprocessed DataFrame
- Data preprocessing and train-test split
- Hyperparameter tuning with cross-validation
- Model comparison across multiple algorithms
- Best model selection and registration to both Model Registry and MLflow
- Comprehensive metrics tracking with MLflow experiments
- Fixed model naming convention for SageMaker Model Registry
- Only best model registered in SageMaker Model Registry

Author: Your Name
Date: July 2025
"""

import boto3
import pandas as pd
import numpy as np
import json
import os
import joblib
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Core ML Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report
)

# SageMaker Libraries
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.model_metrics import MetricsSource, ModelMetrics

# MLflow Libraries
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

class SalesConversionMLPipeline:
    """
    Complete MLOps pipeline for sales conversion prediction with Model Registry and MLflow integration.
    
    This class handles the entire machine learning workflow including:
    - Data preprocessing and train-test split
    - Model training and hyperparameter tuning
    - Model evaluation and comparison
    - Registration of ONLY the best model to SageMaker Model Registry and MLflow
    - Experiment tracking with MLflow
    """
    
    def __init__(self, df, config=None):
        """
        Initialize the ML pipeline with configuration and preprocessed DataFrame.
        
        Args:
            df (pd.DataFrame): Preprocessed DataFrame with features and 'Converted' target
            config (dict): Configuration parameters for the pipeline
        """
        # Default configuration with updated MLflow role
        self.config = config or {
            'region': 'ap-south-1',
            'role_arn': 'arn:aws:iam::394376288194:role/service-role/AmazonSageMaker-ExecutionRole-20250719T144960',
            'bucket_name': 'sagemaker-files-project',
            'model_package_group_name': 'SalesConversionModelGroup',
            'mlflow_tracking_uri': 'arn:aws:sagemaker:ap-south-1:394376288194:mlflow-tracking-server/final-capstone-aravind-mlflow',
            'experiment_name': 'sales-conversion-experiment',
            'test_size': 0.2,
            'random_state': 42,
            'cv_folds': 5
        }
        
        # Set the input DataFrame
        self.data = df.copy()
        
        # Initialize AWS clients
        self.s3_client = boto3.client('s3', region_name=self.config['region'])
        self.sm_client = boto3.client('sagemaker', region_name=self.config['region'])
        self.sagemaker_session = sagemaker.Session()
        
        # Initialize MLflow
        self.setup_mlflow()
        
        # Initialize variables
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.scaler = StandardScaler()
        self.models = {}
        self.model_results = {}
        self.evaluation_results = {}
        self.best_model = None
        self.best_model_name = None
        self.model_artifacts = {}
        self.mlflow_run_ids = {}
        
        print("🚀 Sales Conversion ML Pipeline initialized successfully!")
        print(f"📊 Configuration: {json.dumps(self.config, indent=2)}")
        print(f"📈 Dataset shape: {self.data.shape}")
        print(f"🎯 Target distribution: {self.data['Converted'].value_counts().to_dict()}")
        
    def setup_mlflow(self):
        """
        Setup MLflow tracking with SageMaker Studio integration.
        """
        try:
            print("🔧 Setting up MLflow tracking...")
            
            # Set MLflow tracking URI
            mlflow.set_tracking_uri(self.config['mlflow_tracking_uri'])
            
            # Set experiment
            mlflow.set_experiment(self.config['experiment_name'])
            
            # Initialize MLflow client
            self.mlflow_client = MlflowClient()
            
            print(f"✅ MLflow setup completed!")
            print(f"🔗 Tracking URI: {self.config['mlflow_tracking_uri']}")
            print(f"🧪 Experiment: {self.config['experiment_name']}")
            
        except Exception as e:
            print(f"❌ Error setting up MLflow: {str(e)}")
            raise
    
    def prepare_data(self):
        """
        Prepare data for training including train-test split and scaling.
        
        Returns:
            tuple: (X_train, X_test, y_train, y_test)
        """
        try:
            print("🔧 Preparing data for training...")
            
            if self.data is None:
                raise ValueError("Data not provided. Please provide a preprocessed DataFrame.")
            
            # Check if 'Converted' column exists
            if 'Converted' not in self.data.columns:
                raise ValueError("Target column 'Converted' not found in the DataFrame.")
            
            # Separate features and target
            X = self.data.drop('Converted', axis=1)
            y = self.data['Converted']
            
            # Train-test split
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                X, y, 
                test_size=self.config['test_size'], 
                random_state=self.config['random_state'],
                stratify=y
            )
            
            # Scale features
            self.X_train_scaled = self.scaler.fit_transform(self.X_train)
            self.X_test_scaled = self.scaler.transform(self.X_test)
            
            print(f"✅ Data preparation completed!")
            print(f"🏋️ Training set: {self.X_train.shape[0]} samples")
            print(f"🧪 Test set: {self.X_test.shape[0]} samples")
            print(f"📊 Features: {self.X_train.shape[1]}")
            
            return self.X_train, self.X_test, self.y_train, self.y_test
            
        except Exception as e:
            print(f"❌ Error in data preparation: {str(e)}")
            raise
    
    def define_models_and_hyperparameters(self):
        """
        Define models and their hyperparameter grids for tuning.
        
        Returns:
            dict: Dictionary of models and their hyperparameter grids
        """
        print("🔍 Defining models and hyperparameter grids...")
        
        # Model definitions with hyperparameter grids
        model_configs = {
            'RandomForest': {
                'model': RandomForestClassifier(random_state=self.config['random_state']),
                'params': {
                    'n_estimators': [50, 100, 200],
                    'max_depth': [10, 20, None],
                    'min_samples_split': [2, 5, 10],
                    'min_samples_leaf': [1, 2, 4]
                }
            },
            'GradientBoosting': {
                'model': GradientBoostingClassifier(random_state=self.config['random_state']),
                'params': {
                    'n_estimators': [50, 100, 200],
                    'learning_rate': [0.01, 0.1, 0.2],
                    'max_depth': [3, 5, 7],
                    'subsample': [0.8, 0.9, 1.0]
                }
            },
            'LogisticRegression': {
                'model': LogisticRegression(random_state=self.config['random_state'], max_iter=1000),
                'params': {
                    'C': [0.1, 1, 10, 100],
                    'penalty': ['l2'],
                    'solver': ['liblinear', 'lbfgs']
                }
            },
            'SVM': {
                'model': SVC(random_state=self.config['random_state'], probability=True),
                'params': {
                    'C': [0.1, 1, 10],
                    'kernel': ['rbf', 'linear'],
                    'gamma': ['scale', 'auto']
                }
            }
        }
        
        self.models = model_configs
        print(f"✅ {len(model_configs)} models defined with hyperparameter grids")
        
        return model_configs
    
    def train_and_tune_models(self):
        """
        Train all models with hyperparameter tuning using cross-validation and MLflow tracking.
        
        Returns:
            dict: Results for all trained models
        """
        print("🎯 Starting model training and hyperparameter tuning...")
        
        if not self.models:
            self.define_models_and_hyperparameters()
        
        # Cross-validation strategy
        cv_strategy = StratifiedKFold(n_splits=self.config['cv_folds'], shuffle=True, 
                                     random_state=self.config['random_state'])
        
        for model_name, model_config in self.models.items():
            print(f"\n🔄 Training {model_name}...")
            
            # Start MLflow run for this model
            with mlflow.start_run(run_name=f"{model_name}_training") as run:
                try:
                    # Log model configuration
                    mlflow.log_param("model_name", model_name)
                    mlflow.log_param("cv_folds", self.config['cv_folds'])
                    mlflow.log_param("test_size", self.config['test_size'])
                    mlflow.log_param("random_state", self.config['random_state'])
                    
                    # Log hyperparameter grid
                    for param, values in model_config['params'].items():
                        mlflow.log_param(f"param_grid_{param}", str(values))
                    
                    # Grid search with cross-validation
                    grid_search = GridSearchCV(
                        estimator=model_config['model'],
                        param_grid=model_config['params'],
                        cv=cv_strategy,
                        scoring='roc_auc',
                        n_jobs=-1,
                        verbose=1
                    )
                    
                    # Fit the model
                    grid_search.fit(self.X_train_scaled, self.y_train)
                    
                    # Log best parameters
                    for param, value in grid_search.best_params_.items():
                        mlflow.log_param(f"best_{param}", value)
                    
                    # Log best CV score
                    mlflow.log_metric("best_cv_score", grid_search.best_score_)
                    
                    # Store results
                    self.model_results[model_name] = {
                        'model': grid_search.best_estimator_,
                        'best_params': grid_search.best_params_,
                        'best_cv_score': grid_search.best_score_,
                        'cv_results': grid_search.cv_results_
                    }
                    
                    # Store MLflow run ID
                    self.mlflow_run_ids[model_name] = run.info.run_id
                    
                    print(f"✅ {model_name} training completed!")
                    print(f"🏆 Best CV Score: {grid_search.best_score_:.4f}")
                    print(f"⚙️ Best Parameters: {grid_search.best_params_}")
                    print(f"📝 MLflow Run ID: {run.info.run_id}")
                    
                except Exception as e:
                    print(f"❌ Error training {model_name}: {str(e)}")
                    mlflow.log_param("error", str(e))
                    continue
        
        print(f"\n🎉 Model training completed for {len(self.model_results)} models!")
        return self.model_results
    
    def create_sagemaker_compatible_name(self, model_name):
        """
        Create SageMaker Model Registry compatible name.
        
        Args:
            model_name (str): Original model name
            
        Returns:
            str: SageMaker compatible name
        """
        # Replace underscores and special characters with hyphens
        # SageMaker allows: ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,56}$
        compatible_name = model_name.replace('_', '-').replace(' ', '-')
        compatible_name = 'SalesConversion-' + compatible_name
        
        # Ensure it doesn't exceed 56 characters
        if len(compatible_name) > 56:
            compatible_name = compatible_name[:56]
        
        return compatible_name
    
    def evaluate_models(self):
        """
        Evaluate all trained models on the test set with MLflow logging.
        Only log models to MLflow but don't register them yet - registration happens only for best model.
        
        Returns:
            dict: Comprehensive evaluation results for all models
        """
        print("📊 Evaluating models on test set...")
        
        evaluation_results = {}
        
        for model_name, model_data in self.model_results.items():
            print(f"\n🔍 Evaluating {model_name}...")
            
            # Continue the existing MLflow run for this model
            with mlflow.start_run(run_id=self.mlflow_run_ids[model_name]):
                try:
                    model = model_data['model']
                    
                    # Make predictions
                    y_pred = model.predict(self.X_test_scaled)
                    y_pred_proba = model.predict_proba(self.X_test_scaled)[:, 1]
                    
                    # Calculate metrics
                    metrics = {
                        'accuracy': accuracy_score(self.y_test, y_pred),
                        'precision': precision_score(self.y_test, y_pred, average='weighted'),
                        'recall': recall_score(self.y_test, y_pred, average='weighted'),
                        'f1_score': f1_score(self.y_test, y_pred, average='weighted'),
                        'roc_auc': roc_auc_score(self.y_test, y_pred_proba),
                        'cv_score': model_data['best_cv_score']
                    }
                    
                    # Log metrics to MLflow
                    for metric_name, metric_value in metrics.items():
                        mlflow.log_metric(metric_name, metric_value)
                    
                    # Confusion matrix
                    cm = confusion_matrix(self.y_test, y_pred)
                    
                    # Classification report
                    class_report = classification_report(self.y_test, y_pred, output_dict=True)
                    
                    # Feature importance (if available)
                    feature_importance = None
                    if hasattr(model, 'feature_importances_'):
                        feature_importance = pd.DataFrame({
                            'feature': self.X_train.columns,
                            'importance': model.feature_importances_
                        }).sort_values('importance', ascending=False)
                        
                        # Log top 10 feature importances to MLflow
                        for idx, row in feature_importance.head(10).iterrows():
                            mlflow.log_metric(f"feature_importance_{row['feature']}", row['importance'])
                    
                    # Create SageMaker compatible model name for reference
                    sagemaker_model_name = self.create_sagemaker_compatible_name(model_name)
                    
                    # Log model to MLflow (but don't register yet - only log as artifact)
                    mlflow.sklearn.log_model(
                        sk_model=model,
                        artifact_path="model",
                        signature=mlflow.models.infer_signature(self.X_train_scaled, y_pred_proba)
                    )
                    
                    # Log scaler as artifact
                    scaler_path = f"{model_name}_scaler.pkl"
                    joblib.dump(self.scaler, scaler_path)
                    mlflow.log_artifact(scaler_path)
                    os.remove(scaler_path)  # Clean up
                    
                    evaluation_results[model_name] = {
                        'metrics': metrics,
                        'confusion_matrix': cm,
                        'classification_report': class_report,
                        'feature_importance': feature_importance,
                        'best_params': model_data['best_params'],
                        'mlflow_run_id': self.mlflow_run_ids[model_name],
                        'sagemaker_model_name': sagemaker_model_name
                    }
                    
                    # Print metrics
                    print(f"📈 {model_name} Metrics:")
                    for metric, value in metrics.items():
                        print(f"  {metric}: {value:.4f}")
                    
                    print(f"✅ {model_name} evaluation completed!")
                    print(f"📝 MLflow Run ID: {self.mlflow_run_ids[model_name]}")
                    
                except Exception as e:
                    print(f"❌ Error evaluating {model_name}: {str(e)}")
                    mlflow.log_param("evaluation_error", str(e))
                    print(f"📝 MLflow Run ID: {self.mlflow_run_ids[model_name]}")
                    continue
        
        self.evaluation_results = evaluation_results
        print(f"\n✅ Model evaluation completed for {len(evaluation_results)} models!")
        
        return evaluation_results
    
    def select_best_model(self):
        """
        Select the best model based on ROC-AUC score and log to MLflow.
        
        Returns:
            tuple: (best_model_name, best_model, best_metrics)
        """
        print("🏆 Selecting best model...")
        
        if not hasattr(self, 'evaluation_results') or len(self.evaluation_results) == 0:
            raise ValueError("No models evaluated successfully. Please check model evaluation results.")
        
        # Compare models by ROC-AUC score
        model_scores = {}
        for model_name, results in self.evaluation_results.items():
            model_scores[model_name] = results['metrics']['roc_auc']
        
        if len(model_scores) == 0:
            raise ValueError("No valid model scores found. Pipeline evaluation failed.")
        
        # Select best model
        self.best_model_name = max(model_scores, key=model_scores.get)
        self.best_model = self.model_results[self.best_model_name]['model']
        best_metrics = self.evaluation_results[self.best_model_name]['metrics']
        
        # Log best model selection to MLflow
        with mlflow.start_run(run_name="best_model_selection") as run:
            mlflow.log_param("best_model_name", self.best_model_name)
            mlflow.log_metric("best_roc_auc", best_metrics['roc_auc'])
            mlflow.log_metric("best_accuracy", best_metrics['accuracy'])
            mlflow.log_metric("best_f1_score", best_metrics['f1_score'])
            
            # Log model comparison
            for model_name, score in model_scores.items():
                mlflow.log_metric(f"roc_auc_{model_name}", score)
        
        print(f"🥇 Best Model: {self.best_model_name}")
        print(f"🎯 Best ROC-AUC: {best_metrics['roc_auc']:.4f}")
        print(f"⚙️ Best Parameters: {self.evaluation_results[self.best_model_name]['best_params']}")
        
        # Print comparison table
        print("\n📊 Model Comparison:")
        print("-" * 80)
        print(f"{'Model':<20} {'ROC-AUC':<10} {'Accuracy':<10} {'F1-Score':<10} {'CV Score':<10}")
        print("-" * 80)
        
        for model_name, results in self.evaluation_results.items():
            metrics = results['metrics']
            print(f"{model_name:<20} {metrics['roc_auc']:<10.4f} {metrics['accuracy']:<10.4f} "
                  f"{metrics['f1_score']:<10.4f} {metrics['cv_score']:<10.4f}")
        
        return self.best_model_name, self.best_model, best_metrics
    
    def prepare_model_artifacts(self):
        """
        Prepare model artifacts for registration (only for the best model).
        
        Returns:
            dict: Model artifacts information
        """
        print("📦 Preparing model artifacts for the best model...")
        
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # Create model artifacts (only for best model)
            self.model_artifacts = {
                'model': self.best_model,
                'scaler': self.scaler,
                'feature_names': list(self.X_train.columns),
                'model_name': self.best_model_name,
                'best_params': self.evaluation_results[self.best_model_name]['best_params'],
                'metrics': self.evaluation_results[self.best_model_name]['metrics'],
                'timestamp': timestamp,
                'training_data_shape': self.X_train.shape,
                'test_data_shape': self.X_test.shape,
                'mlflow_run_id': self.mlflow_run_ids[self.best_model_name]
            }
            
            # Save model artifacts locally
            artifacts_filename = f'best_model_artifacts_{timestamp}.joblib'
            joblib.dump(self.model_artifacts, artifacts_filename)
            
            # Upload to S3
            s3_key = f'model_artifacts/best_model/{artifacts_filename}'
            self.s3_client.upload_file(artifacts_filename, self.config['bucket_name'], s3_key)
            
            model_data_url = f"s3://{self.config['bucket_name']}/{s3_key}"
            
            # Clean up local file
            os.remove(artifacts_filename)
            
            print(f"✅ Best model artifacts prepared and uploaded to: {model_data_url}")
            
            return {
                'model_data_url': model_data_url,
                'artifacts_key': s3_key,
                'timestamp': timestamp
            }
            
        except Exception as e:
            print(f"❌ Error preparing model artifacts: {str(e)}")
            raise
    
    def create_inference_code(self):
        """
        Create inference code for the best model.
        
        Returns:
            str: S3 URL of the inference code
        """
        print("💻 Creating inference code for the best model...")
        
        try:
            # Create inference script
            inference_script = f'''
import joblib
import json
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

def model_fn(model_dir):
    """Load model artifacts"""
    import os
    model_path = os.path.join(model_dir, 'best_model_artifacts.joblib')
    model_artifacts = joblib.load(model_path)
    return model_artifacts

def input_fn(request_body, request_content_type):
    """Parse input data"""
    if request_content_type == 'application/json':
        input_data = json.loads(request_body)
        return pd.DataFrame(input_data)
    elif request_content_type == 'text/csv':
        from io import StringIO
        return pd.read_csv(StringIO(request_body))
    else:
        raise ValueError(f"Unsupported content type: {{request_content_type}}")

def predict_fn(input_data, model_artifacts):
    """Make predictions using the best model"""
    model = model_artifacts['model']
    scaler = model_artifacts['scaler']
    
    # Ensure input data has the correct features
    expected_features = model_artifacts['feature_names']
    if list(input_data.columns) != expected_features:
        # Reorder columns to match training data
        input_data = input_data[expected_features]
    
    # Scale input data
    input_scaled = scaler.transform(input_data)
    
    # Make predictions
    predictions = model.predict(input_scaled)
    probabilities = model.predict_proba(input_scaled)[:, 1]
    
    # Create lead categories
    lead_categories = np.where(probabilities >= 0.7, 'High', 
                             np.where(probabilities >= 0.4, 'Medium', 'Low'))
    
    return {{
        'predictions': predictions.tolist(),
        'probabilities': probabilities.tolist(),
        'lead_categories': lead_categories.tolist(),
        'model_used': model_artifacts['model_name']
    }}

def output_fn(prediction, content_type):
    """Format output"""
    if content_type == 'application/json':
        return json.dumps(prediction)
    else:
        raise ValueError(f"Unsupported content type: {{content_type}}")
'''
            
            # Save inference script
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            script_filename = f'best_model_inference_{timestamp}.py'
            
            with open(script_filename, 'w') as f:
                f.write(inference_script)
            
            # Upload to S3
            s3_key = f'inference_code/best_model/{script_filename}'
            self.s3_client.upload_file(script_filename, self.config['bucket_name'], s3_key)
            
            inference_code_url = f"s3://{self.config['bucket_name']}/{s3_key}"
            
            # Clean up local file
            os.remove(script_filename)
            
            print(f"✅ Best model inference code uploaded to: {inference_code_url}")
            
            return inference_code_url
            
        except Exception as e:
            print(f"❌ Error creating inference code: {str(e)}")
            raise
    
    def register_best_model_in_mlflow(self):
        """
        Register ONLY the best model in MLflow Model Registry.
        
        Returns:
            dict: Model registration information
        """
        print("📋 Registering ONLY the best model in MLflow Model Registry...")
        
        try:
            # Ensure we have a best model selected
            if not hasattr(self, 'best_model') or self.best_model is None:
                raise ValueError("No best model found. Please run model selection first.")
            
            # Get best model info
            best_model = self.best_model
            best_model_name = self.best_model_name
            best_metrics = self.evaluation_results[best_model_name]['metrics']
            best_params = self.evaluation_results[best_model_name]['best_params']
            
            # Create MLflow compatible model name
            mlflow_model_name = self.create_sagemaker_compatible_name(best_model_name)
            
            # Create timestamp
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # Start or continue MLflow run for the best model
            with mlflow.start_run(run_id=self.mlflow_run_ids[best_model_name]):
                
                # Create model signature
                signature = mlflow.models.infer_signature(
                    self.X_train_scaled, 
                    best_model.predict_proba(self.X_train_scaled)[:, 1]
                )
                
                # Register the best model in MLflow Model Registry
                model_info = mlflow.sklearn.log_model(
                    sk_model=best_model,
                    artifact_path="best_model",
                    registered_model_name=mlflow_model_name,
                    signature=signature,
                    input_example=self.X_train_scaled[:5],  # First 5 samples as example
                    metadata={
                        "model_type": "Sales Conversion Classifier",
                        "algorithm": best_model_name,
                        "training_timestamp": timestamp,
                        "roc_auc_score": best_metrics['roc_auc'],
                        "accuracy": best_metrics['accuracy'],
                        "best_parameters": str(best_params)
                    }
                )
                
                # Log additional artifacts
                # Save and log the scaler
                scaler_path = f"best_model_scaler.pkl"
                joblib.dump(self.scaler, scaler_path)
                mlflow.log_artifact(scaler_path, artifact_path="preprocessing")
                os.remove(scaler_path)  # Clean up
                
                # Save and log feature names
                feature_names_path = f"feature_names.json"
                with open(feature_names_path, 'w') as f:
                    json.dump(list(self.X_train.columns), f)
                mlflow.log_artifact(feature_names_path, artifact_path="preprocessing")
                os.remove(feature_names_path)  # Clean up
                
                # Log model metadata
                mlflow.log_param("is_best_model", True)
                mlflow.log_param("model_registry_status", "registered_as_best")
                mlflow.log_param("mlflow_model_name", mlflow_model_name)
                mlflow.log_param("registration_timestamp", timestamp)
                
                # Log best model metrics again for emphasis
                mlflow.log_metric("best_model_roc_auc", best_metrics['roc_auc'])
                mlflow.log_metric("best_model_accuracy", best_metrics['accuracy'])
                mlflow.log_metric("best_model_f1_score", best_metrics['f1_score'])
                
                # Log model version info
                mlflow.log_param("model_version", model_info.registered_model_version)
                
            # Get the registered model version
            client = MlflowClient()
            model_version = client.get_latest_versions(mlflow_model_name, stages=["None"])[0]
            
            # Update model version description
            client.update_model_version(
                name=mlflow_model_name,
                version=model_version.version,
                description=f"Best performing {best_model_name} model with ROC-AUC: {best_metrics['roc_auc']:.4f} and Accuracy: {best_metrics['accuracy']:.4f}. Trained on {timestamp}."
            )
            
            # Prepare registration info
            registration_info = {
                'mlflow_model_name': mlflow_model_name,
                'model_version': model_version.version,
                'model_uri': f"models:/{mlflow_model_name}/{model_version.version}",
                'model_name': best_model_name,
                'timestamp': timestamp,
                'metrics': best_metrics,
                'best_params': best_params,
                'mlflow_run_id': self.mlflow_run_ids[best_model_name],
                'registration_type': 'MLflow_Model_Registry',
                'model_stage': 'None'  # Initial stage
            }
            
            print(f"✅ BEST model registered successfully in MLflow Model Registry!")
            print(f"📦 MLflow Model Name: {mlflow_model_name}")
            print(f"📊 Model Version: {model_version.version}")
            print(f"🔗 Model URI: models:/{mlflow_model_name}/{model_version.version}")
            print(f"🎯 Algorithm: {best_model_name}")
            print(f"🏆 ROC-AUC Score: {best_metrics['roc_auc']:.4f}")
            print(f"🎪 Accuracy: {best_metrics['accuracy']:.4f}")
            print(f"📝 MLflow Run ID: {self.mlflow_run_ids[best_model_name]}")
            print(f"⭐ Registration Status: Completed")
            
            return registration_info
            
        except Exception as e:
            print(f"❌ Error registering best model in MLflow: {str(e)}")
            raise

    def register_best_model_only(self):
        """
        Register ONLY the best model in MLflow Model Registry.
        Removed SageMaker Model Registry registration due to ECR access issues.
        
        Returns:
            dict: Model registration information
        """
        print("📋 Registering ONLY the best model in MLflow Model Registry...")
        
        try:
            # Prepare model artifacts
            artifacts_info = self.prepare_model_artifacts()
            
            # Create inference code
            inference_code_url = self.create_inference_code()
            
            # Prepare model metrics
            metrics = self.evaluation_results[self.best_model_name]['metrics']
            
            # Create timestamp
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # Register ONLY the best model in MLflow Model Registry
            sagemaker_model_name = self.evaluation_results[self.best_model_name]['sagemaker_model_name']
            
            # Register the best model in MLflow with the proper name
            with mlflow.start_run(run_id=self.mlflow_run_ids[self.best_model_name]):
                # Register the best model in MLflow Model Registry
                model_version = mlflow.sklearn.log_model(
                    sk_model=self.best_model,
                    artifact_path="best_model",
                    registered_model_name=sagemaker_model_name,
                    signature=mlflow.models.infer_signature(self.X_train_scaled, 
                                                           self.best_model.predict_proba(self.X_train_scaled)[:, 1])
                )
                
                # Log that this is the best model
                mlflow.log_param("is_best_model", True)
                mlflow.log_param("model_registry_status", "registered_as_best_in_mlflow")
                mlflow.log_param("registration_timestamp", timestamp)
            
            registration_info = {
                'mlflow_model_name': sagemaker_model_name,
                'model_name': self.best_model_name,
                'model_data_url': artifacts_info['model_data_url'],
                'inference_code_url': inference_code_url,
                'timestamp': timestamp,
                'metrics': metrics,
                'best_params': self.evaluation_results[self.best_model_name]['best_params'],
                'mlflow_run_id': self.mlflow_run_ids[self.best_model_name],
                'registration_type': 'MLflow_Only'
            }
            
            print(f"✅ BEST model registered successfully in MLflow!")
            print(f"📦 MLflow Model Name: {sagemaker_model_name}")
            print(f"🎯 Model Name: {self.best_model_name}")
            print(f"🏆 ROC-AUC Score: {metrics['roc_auc']:.4f}")
            print(f"🎪 Accuracy: {metrics['accuracy']:.4f}")
            print(f"📝 MLflow Run ID: {self.mlflow_run_ids[self.best_model_name]}")
            print(f"⚠️ SageMaker Model Registry registration skipped due to ECR access issues")
            
            return registration_info
            
        except Exception as e:
            print(f"❌ Error registering best model: {str(e)}")
            raise
    
    def save_model_metrics_to_s3(self, metrics, timestamp):
        """
        Save model metrics to S3 for Model Registry.
        
        Args:
            metrics (dict): Model metrics
            timestamp (str): Timestamp string
            
        Returns:
            str: S3 URI of the metrics file
        """
        try:
            # Create metrics in SageMaker format
            sagemaker_metrics = {
                "version": "1.0",
                "metrics": {
                    "accuracy": {
                        "value": metrics['accuracy'],
                        "standard_deviation": 0.0
                    },
                    "precision": {
                        "value": metrics['precision'],
                        "standard_deviation": 0.0
                    },
                    "recall": {
                        "value": metrics['recall'],
                        "standard_deviation": 0.0
                    },
                    "f1_score": {
                        "value": metrics['f1_score'],
                        "standard_deviation": 0.0
                    },
                    "roc_auc": {
                        "value": metrics['roc_auc'],
                        "standard_deviation": 0.0
                    }
                }
            }
            
            # Save metrics file
            metrics_filename = f'best_model_metrics_{timestamp}.json'
            with open(metrics_filename, 'w') as f:
                json.dump(sagemaker_metrics, f, indent=2)
            
            # Upload to S3
            s3_key = f'model_metrics/best_model/{metrics_filename}'
            self.s3_client.upload_file(metrics_filename, self.config['bucket_name'], s3_key)
            
            metrics_s3_uri = f"s3://{self.config['bucket_name']}/{s3_key}"
            
            # Clean up local file
            os.remove(metrics_filename)
            
            return metrics_s3_uri
            
        except Exception as e:
            print(f"❌ Error saving metrics to S3: {str(e)}")
            raise
    
    def run_complete_pipeline(self):
        """
        Run the complete ML pipeline from data preparation to best model registration.
        
        Returns:
            dict: Complete pipeline results
        """
        print("🔄 Starting complete ML pipeline...")
        print("=" * 60)
        
        try:
            # Step 1: Prepare data
            print("\n🔧 STEP 1: Preparing data...")
            self.prepare_data()
            
            # Step 2: Train and tune models
            print("\n🎯 STEP 2: Training and tuning models...")
            self.train_and_tune_models()
            
            # Step 3: Evaluate models
            print("\n📊 STEP 3: Evaluating models...")
            self.evaluate_models()
            
            # Step 4: Select best model
            print("\n🏆 STEP 4: Selecting best model...")
            best_model_info = self.select_best_model()
            
            # Step 5: Register ONLY the best model in MLflow registry
            print("\n📋 STEP 5: Registering ONLY the best model in MLflow Model Registry...")
            registration_info = self.register_best_model_only()
            
            # Compile complete results
            pipeline_results = {
                'data_info': {
                    'dataset_shape': self.data.shape,
                    'training_samples': self.X_train.shape[0],
                    'test_samples': self.X_test.shape[0],
                    'features': self.X_train.shape[1]
                },
                'best_model': {
                    'name': self.best_model_name,
                    'metrics': self.evaluation_results[self.best_model_name]['metrics'],
                    'parameters': self.evaluation_results[self.best_model_name]['best_params'],
                    'mlflow_run_id': self.mlflow_run_ids[self.best_model_name]
                },
                'all_models_performance': {
                    name: {
                        'metrics': results['metrics'],
                        'mlflow_run_id': results['mlflow_run_id']
                    } for name, results in self.evaluation_results.items()
                },
                'model_registry': registration_info,
                'mlflow_info': {
                    'tracking_uri': self.config['mlflow_tracking_uri'],
                    'experiment_name': self.config['experiment_name'],
                    'run_ids': self.mlflow_run_ids
                },
                'pipeline_timestamp': datetime.now().isoformat()
            }
            
            print("\n" + "=" * 60)
            print("🎉 PIPELINE COMPLETED SUCCESSFULLY!")
            print("=" * 60)
            print(f"🥇 Best Model: {self.best_model_name}")
            print(f"🎯 Best AUC Score: {self.evaluation_results[self.best_model_name]['metrics']['roc_auc']:.4f}")
            print(f"🎪 Accuracy: {self.evaluation_results[self.best_model_name]['metrics']['accuracy']:.4f}")
            print(f"📦 MLflow Model Name: {registration_info['mlflow_model_name']}")
            print(f"📝 MLflow Run ID: {self.mlflow_run_ids[self.best_model_name]}")
            print(f"🔗 MLflow Tracking URI: {self.config['mlflow_tracking_uri']}")
            print("🔥 ONLY THE BEST MODEL HAS BEEN REGISTERED IN MLFLOW MODEL REGISTRY")
            print("⚠️ SageMaker Model Registry registration disabled due to ECR access issues")
            
            return pipeline_results
            
        except Exception as e:
            print(f"❌ Pipeline failed: {str(e)}")
            raise
    
    def generate_model_report(self):
        """
        Generate a comprehensive model report with MLflow integration.
        
        Returns:
            dict: Detailed model report
        """
        print("📋 Generating comprehensive model report...")
        
        if not hasattr(self, 'evaluation_results') or len(self.evaluation_results) == 0:
            raise ValueError("Models not evaluated. Please run the pipeline first.")
        
        report = {
            'executive_summary': {
                'best_model': self.best_model_name,
                'best_auc_score': self.evaluation_results[self.best_model_name]['metrics']['roc_auc'],
                'best_accuracy': self.evaluation_results[self.best_model_name]['metrics']['accuracy'],
                'data_size': self.data.shape[0],
                'feature_count': self.X_train.shape[1],
                'generation_timestamp': datetime.now().isoformat(),
                'mlflow_experiment': self.config['experiment_name'],
                'best_model_run_id': self.mlflow_run_ids[self.best_model_name],
                'registry_status': 'ONLY BEST MODEL REGISTERED IN MLFLOW'
            },
            'data_overview': {
                'total_samples': self.data.shape[0],
                'training_samples': self.X_train.shape[0],
                'test_samples': self.X_test.shape[0],
                'feature_count': self.X_train.shape[1],
                'target_distribution': self.data['Converted'].value_counts().to_dict(),
                'class_balance': self.data['Converted'].value_counts(normalize=True).to_dict()
            },
            'model_comparison': {},
            'best_model_details': {
                'name': self.best_model_name,
                'parameters': self.evaluation_results[self.best_model_name]['best_params'],
                'metrics': self.evaluation_results[self.best_model_name]['metrics'],
                'confusion_matrix': self.evaluation_results[self.best_model_name]['confusion_matrix'].tolist(),
                'classification_report': self.evaluation_results[self.best_model_name]['classification_report'],
                'mlflow_run_id': self.mlflow_run_ids[self.best_model_name],
                'registered_in_sagemaker': True
            },
            'feature_importance': None,
            'mlflow_info': {
                'tracking_uri': self.config['mlflow_tracking_uri'],
                'experiment_name': self.config['experiment_name'],
                'all_run_ids': self.mlflow_run_ids
            },
            'recommendations': []
        }
        
        # Add model comparison with MLflow info
        for model_name, results in self.evaluation_results.items():
            report['model_comparison'][model_name] = {
                'metrics': results['metrics'],
                'best_params': results['best_params'],
                'mlflow_run_id': results['mlflow_run_id'],
                'registered_in_sagemaker': False,
                'registered_in_mlflow': model_name == self.best_model_name
            }
        
        # Add feature importance if available
        if self.evaluation_results[self.best_model_name]['feature_importance'] is not None:
            report['feature_importance'] = self.evaluation_results[self.best_model_name]['feature_importance'].to_dict('records')
        
        # Generate recommendations
        best_auc = report['executive_summary']['best_auc_score']
        best_accuracy = report['executive_summary']['best_accuracy']
        
        if best_auc >= 0.85:
            report['recommendations'].append("🟢 Model performance is excellent (AUC ≥ 0.85). Ready for production deployment.")
        elif best_auc >= 0.75:
            report['recommendations'].append("🟡 Model performance is good (AUC ≥ 0.75). Consider A/B testing before full deployment.")
        else:
            report['recommendations'].append("🔴 Model performance needs improvement (AUC < 0.75). Consider feature engineering or more data.")
        
        if best_accuracy >= 0.80:
            report['recommendations'].append("✅ High accuracy achieved. Monitor for concept drift in production.")
        else:
            report['recommendations'].append("⚠️ Consider ensemble methods or advanced feature engineering to improve accuracy.")
        
        # Check class imbalance
        target_dist = report['data_overview']['class_balance']
        min_class_ratio = min(target_dist.values())
        if min_class_ratio < 0.3:
            report['recommendations'].append("⚖️ Class imbalance detected. Consider SMOTE or class weight adjustments.")
        
        report['recommendations'].append(f"📝 View detailed experiment tracking at: {self.config['mlflow_tracking_uri']}")
        report['recommendations'].append("🔥 Only the best performing model has been registered in MLflow Model Registry")
        report['recommendations'].append("⚠️ SageMaker Model Registry registration disabled due to ECR access issues")
        
        print("✅ Model report generated successfully!")
        return report
    
    def get_mlflow_experiment_info(self):
        """
        Get MLflow experiment information and run details.
        
        Returns:
            dict: MLflow experiment information
        """
        print("📊 Retrieving MLflow experiment information...")
        
        try:
            # Get experiment
            experiment = mlflow.get_experiment_by_name(self.config['experiment_name'])
            
            # Get all runs for this experiment
            runs = mlflow.search_runs(
                experiment_ids=[experiment.experiment_id],
                order_by=["start_time DESC"]
            )
            
            experiment_info = {
                'experiment_id': experiment.experiment_id,
                'experiment_name': experiment.name,
                'artifact_location': experiment.artifact_location,
                'lifecycle_stage': experiment.lifecycle_stage,
                'total_runs': len(runs),
                'recent_runs': []
            }
            
            # Add recent run details
            for _, run in runs.head(10).iterrows():
                run_info = {
                    'run_id': run['run_id'],
                    'run_name': run.get('tags.mlflow.runName', 'Unnamed'),
                    'status': run['status'],
                    'start_time': run['start_time'],
                    'end_time': run['end_time'],
                    'metrics': {col: run[col] for col in run.index if col.startswith('metrics.')},
                    'params': {col: run[col] for col in run.index if col.startswith('params.')}
                }
                experiment_info['recent_runs'].append(run_info)
            
            print(f"✅ Found {len(runs)} runs in experiment '{experiment.name}'")
            return experiment_info
            
        except Exception as e:
            print(f"❌ Error retrieving MLflow experiment info: {str(e)}")
            return None
    
    def compare_models_in_mlflow(self):
        """
        Compare all models using MLflow metrics.
        
        Returns:
            pd.DataFrame: Model comparison DataFrame
        """
        print("📈 Comparing models using MLflow data...")
        
        try:
            # Get experiment
            experiment = mlflow.get_experiment_by_name(self.config['experiment_name'])
            
            # Get runs for our models
            runs_df = mlflow.search_runs(
                experiment_ids=[experiment.experiment_id],
                filter_string="params.model_name != ''"
            )
            
            if runs_df.empty:
                print("No model runs found in MLflow.")
                return None
            
            # Create comparison DataFrame
            comparison_data = []
            for _, run in runs_df.iterrows():
                if 'params.model_name' in run and pd.notna(run['params.model_name']):
                    comparison_data.append({
                        'Model': run['params.model_name'],
                        'Run_ID': run['run_id'],
                        'ROC_AUC': run.get('metrics.roc_auc', 0),
                        'Accuracy': run.get('metrics.accuracy', 0),
                        'F1_Score': run.get('metrics.f1_score', 0),
                        'Precision': run.get('metrics.precision', 0),
                        'Recall': run.get('metrics.recall', 0),
                        'CV_Score': run.get('metrics.best_cv_score', 0),
                        'Start_Time': run['start_time'],
                        'Registered_In_SageMaker': False,
                    'Registered_In_MLflow': run['params.model_name'] == self.best_model_name if hasattr(self, 'best_model_name') else False
                    })
            
            comparison_df = pd.DataFrame(comparison_data)
            comparison_df = comparison_df.sort_values('ROC_AUC', ascending=False)
            
            print("✅ Model comparison DataFrame created from MLflow data")
            return comparison_df
            
        except Exception as e:
            print(f"❌ Error comparing models in MLflow: {str(e)}")
            return None


# Example usage and main execution
if __name__ == "__main__":
    """
    Example usage of the Sales Conversion ML Pipeline with MLflow integration.
    Only the best model will be registered in SageMaker Model Registry.
    """
    
    # Example: Load your preprocessed DataFrame
    # Replace this with your actual preprocessed DataFrame
    print("📊 Loading preprocessed data...")
    
    # For demonstration - replace with your actual DataFrame loading
    # df = pd.read_csv('pre_processed/processed/preprocessed_data.csv')
    
    # Sample data creation for demonstration (replace with your actual data)
    np.random.seed(42)
    n_samples = 1000
    df = pd.DataFrame({
        'feature_1': np.random.normal(0, 1, n_samples),
        'feature_2': np.random.normal(0, 1, n_samples),
        'feature_3': np.random.uniform(0, 1, n_samples),
        'feature_4': np.random.exponential(1, n_samples),
        'feature_5': np.random.gamma(2, 2, n_samples),
        'Converted': np.random.binomial(1, 0.3, n_samples)
    })
    
    print(f"📈 Sample data created with shape: {df.shape}")
    print(f"🎯 Target distribution: {df['Converted'].value_counts().to_dict()}")
    
    # Custom configuration with updated MLflow role
    custom_config = {
        'region': 'ap-south-1',
        'role_arn': 'arn:aws:iam::394376288194:role/service-role/AmazonSageMaker-ExecutionRole-20250719T144960',
        'bucket_name': 'sagemaker-files-project',
        'model_package_group_name': 'SalesConversionModelGroup',
        'mlflow_tracking_uri': 'arn:aws:sagemaker:ap-south-1:394376288194:mlflow-tracking-server/final-capstone-aravind-mlflow',
        'experiment_name': 'sales-conversion-experiment',
        'test_size': 0.2,
        'random_state': 42,
        'cv_folds': 5
    }
    
    # Initialize pipeline with DataFrame
    pipeline = SalesConversionMLPipeline(df=df, config=custom_config)
    
    # Run complete pipeline
    print("🚀 Starting Sales Conversion ML Pipeline with MLflow...")
    print("🔥 ONLY THE BEST MODEL WILL BE REGISTERED IN MLFLOW MODEL REGISTRY")
    print("⚠️ SageMaker Model Registry disabled due to ECR access issues")
    results = pipeline.run_complete_pipeline()
    
    # Generate comprehensive report
    print("\n📋 Generating model report...")
    report = pipeline.generate_model_report()
    
    # Get MLflow experiment information
    print("\n📊 Retrieving MLflow experiment information...")
    experiment_info = pipeline.get_mlflow_experiment_info()
    
    # Compare models using MLflow data
    print("\n📈 Comparing models using MLflow...")
    comparison_df = pipeline.compare_models_in_mlflow()
    if comparison_df is not None:
        print("\n📊 MLflow Model Comparison:")
        print(comparison_df.to_string(index=False))
    
    # Print summary
    print("\n" + "=" * 60)
    print("📊 PIPELINE SUMMARY")
    print("=" * 60)
    print(f"🎯 Best Model: {results['best_model']['name']}")
    print(f"🏆 AUC Score: {results['best_model']['metrics']['roc_auc']:.4f}")
    print(f"🎪 Accuracy: {results['best_model']['metrics']['accuracy']:.4f}")
    print(f"📦 MLflow Model Name: {results['model_registry']['mlflow_model_name']}")
    print(f"💾 Model Data URL: {results['model_registry']['model_data_url']}")
    print(f"📝 MLflow Run ID: {results['best_model']['mlflow_run_id']}")
    print(f"🔗 MLflow Tracking URI: {results['mlflow_info']['tracking_uri']}")
    print(f"🧪 MLflow Experiment: {results['mlflow_info']['experiment_name']}")
    print("🔥 ONLY THE BEST MODEL HAS BEEN REGISTERED IN MLFLOW MODEL REGISTRY")
    print("⚠️ SageMaker Model Registry registration disabled due to ECR access issues")
    
    # Save complete results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_data = {
        'pipeline_results': results,
        'model_report': report,
        'mlflow_experiment_info': experiment_info,
        'model_comparison': comparison_df.to_dict('records') if comparison_df is not None else None
    }
    
    with open(f'pipeline_results_mlflow_only_{timestamp}.json', 'w') as f:
        json.dump(results_data, f, indent=2, default=str)
    
    print("\n✅ Pipeline execution completed successfully!")
    print(f"📁 Results saved to pipeline_results_mlflow_only_{timestamp}.json")
    
    # Display final model comparison table
    print("\n📊 Final Model Comparison (🔥 = Registered in MLflow):")
    print("-" * 110)
    print(f"{'Model':<20} {'ROC-AUC':<10} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'MLflow':<10}")
    print("-" * 110)
    
    for model_name, model_data in results['all_models_performance'].items():
        metrics = model_data['metrics']
        is_best = "🔥 YES" if model_name == results['best_model']['name'] else "No"
        print(f"{model_name:<20} {metrics['roc_auc']:<10.4f} {metrics['accuracy']:<10.4f} "
              f"{metrics['precision']:<10.4f} {metrics['recall']:<10.4f} {metrics['f1_score']:<10.4f} {is_best:<10}")
    
    print("\n🎉 All operations completed successfully!")
    
    # MLflow specific information
    print("\n" + "=" * 60)
    print("🔬 MLFLOW INTEGRATION SUMMARY")
    print("=" * 60)
    print(f"📊 Experiment Name: {custom_config['experiment_name']}")
    print(f"🔗 Tracking URI: {custom_config['mlflow_tracking_uri']}")
    print(f"🏃 Total Runs Created: {len(pipeline.mlflow_run_ids)}")
    print("📝 All models logged to MLflow, but only best model registered in MLflow Model Registry")
    print("\n📝 Run IDs for each model:")
    for model_name, run_id in pipeline.mlflow_run_ids.items():
        is_best = " (🔥 REGISTERED IN MLFLOW MODEL REGISTRY)" if model_name == pipeline.best_model_name else ""
        print(f"  {model_name}: {run_id}{is_best}")
    
    print("\n" + "=" * 60)
    print("🔧 KEY CHANGES MADE:")
    print("=" * 60)
    print("✅ Removed SageMaker Model Registry registration (ECR access issues)")
    print("✅ Only best model registered in MLflow Model Registry")
    print("✅ All models still tracked in MLflow for comparison")
    print("✅ Enhanced artifact organization for best model")
    print("✅ Clear distinction between MLflow tracking and MLflow model registration")
    print("⚠️ SageMaker deployment will need to be done separately with proper ECR setup")
    print("=" * 60)
    
    # Instructions for accessing MLflow UI
    print("\n📋 TO ACCESS MLFLOW UI:")
    print("=" * 30)
    print("1. Go to SageMaker Studio")
    print("2. Navigate to MLflow in the left sidebar")
    print("3. Open your tracking server: final-capstone-aravind-mlflow")
    print("4. View experiment: sales-conversion-experiment")
    print("5. Compare runs and models interactively")
    print("6. 🔥 Best model is registered in MLflow Model Registry")
    print("7. Access registered models in the 'Models' tab")
    print("=" * 30)
    
    # MLflow Model Registry instructions
    print("\n📦 MLFLOW MODEL REGISTRY ACCESS:")
    print("=" * 40)
    print("1. In MLflow UI, go to 'Models' tab")
    print("2. Find your registered model: SalesConversion-[BestModelName]")
    print("3. View model versions and metadata")
    print("4. Download model artifacts if needed")
    print("5. Use model for inference or deployment")
    print("=" * 40)
    
    # Troubleshooting section
    print("\n🔧 TROUBLESHOOTING & NEXT STEPS:")
    print("=" * 40)
    print("✅ Pipeline now works without ECR access issues")
    print("✅ Best model safely stored in MLflow Model Registry")
    print("📝 For SageMaker deployment:")
    print("  - Set up proper ECR repository access")
    print("  - Use custom container images")
    print("  - Or deploy using MLflow models directly")
    print("📝 For production deployment:")
    print("  - Use MLflow model serving capabilities")
    print("  - Or export model from MLflow to your preferred platform")
    print("=" * 40)
    
    # Display final model comparison table
    print("\n📊 Final Model Comparison (🔥 = Registered in SageMaker):")
    print("-" * 110)
    print(f"{'Model':<20} {'ROC-AUC':<10} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'SageMaker':<10}")
    print("-" * 110)
    
    for model_name, model_data in results['all_models_performance'].items():
        metrics = model_data['metrics']
        is_best = "🔥 YES" if model_name == results['best_model']['name'] else "No"
        print(f"{model_name:<20} {metrics['roc_auc']:<10.4f} {metrics['accuracy']:<10.4f} "
              f"{metrics['precision']:<10.4f} {metrics['recall']:<10.4f} {metrics['f1_score']:<10.4f} {is_best:<10}")
    
    print("\n🎉 All operations completed successfully!")
    
    # MLflow specific information
    print("\n" + "=" * 60)
    print("🔬 MLFLOW INTEGRATION SUMMARY")
    print("=" * 60)
    print(f"📊 Experiment Name: {custom_config['experiment_name']}")
    print(f"🔗 Tracking URI: {custom_config['mlflow_tracking_uri']}")
    print(f"🏃 Total Runs Created: {len(pipeline.mlflow_run_ids)}")
    print("📝 All models logged to MLflow, but only best model registered in SageMaker")
    print("\n📝 Run IDs for each model:")
    for model_name, run_id in pipeline.mlflow_run_ids.items():
        is_best = " (🔥 REGISTERED IN SAGEMAKER)" if model_name == pipeline.best_model_name else ""
        print(f"  {model_name}: {run_id}{is_best}")
    
    print("\n" + "=" * 60)
    print("🔧 KEY CHANGES MADE:")
    print("=" * 60)
    print("✅ Removed model package group creation code")
    print("✅ Only best model registered in SageMaker Model Registry")
    print("✅ All models still tracked in MLflow for comparison")
    print("✅ Enhanced artifact organization for best model")
    print("✅ Clear distinction between MLflow tracking and SageMaker registration")
    print("=" * 60)
    
    # Instructions for accessing MLflow UI
    print("\n📋 TO ACCESS MLFLOW UI:")
    print("=" * 30)
    print("1. Go to SageMaker Studio")
    print("2. Navigate to MLflow in the left sidebar")
    print("3. Open your tracking server: final-capstone-aravind-mlflow")
    print("4. View experiment: sales-conversion-experiment")
    print("5. Compare runs and models interactively")
    print("6. 🔥 Best model is marked and registered in SageMaker")
    print("=" * 30)

📊 Loading preprocessed data...
📈 Sample data created with shape: (1000, 6)
🎯 Target distribution: {0: 737, 1: 263}
🔧 Setting up MLflow tracking...
✅ MLflow setup completed!
🔗 Tracking URI: arn:aws:sagemaker:ap-south-1:394376288194:mlflow-tracking-server/final-capstone-aravind-mlflow
🧪 Experiment: sales-conversion-experiment
🚀 Sales Conversion ML Pipeline initialized successfully!
📊 Configuration: {
  "region": "ap-south-1",
  "role_arn": "arn:aws:iam::394376288194:role/service-role/AmazonSageMaker-ExecutionRole-20250719T144960",
  "bucket_name": "sagemaker-files-project",
  "model_package_group_name": "SalesConversionModelGroup",
  "mlflow_tracking_uri": "arn:aws:sagemaker:ap-south-1:394376288194:mlflow-tracking-server/final-capstone-aravind-mlflow",
  "experiment_name": "sales-conversion-experiment",
  "test_size": 0.2,
  "random_state": 42,
  "cv_folds": 5
}
📈 Dataset shape: (1000, 6)
🎯 Target distribution: {0: 737, 1: 263}
🚀 Starting Sales Conversion ML Pipeline with MLflow...
🔥 ONL

Registered model 'SalesConversion-LogisticRegression' already exists. Creating a new version of this model...
2025/07/19 17:37:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SalesConversion-LogisticRegression, version 6
Created version '6' of model 'SalesConversion-LogisticRegression'.


🏃 View run LogisticRegression_training at: https://ap-south-1.experiments.sagemaker.aws/#/experiments/1/runs/2aa45455ac674c838fd6d795698e8336
🧪 View experiment at: https://ap-south-1.experiments.sagemaker.aws/#/experiments/1
✅ BEST model registered successfully in MLflow!
📦 MLflow Model Name: SalesConversion-LogisticRegression
🎯 Model Name: LogisticRegression
🏆 ROC-AUC Score: 0.5237
🎪 Accuracy: 0.7350
📝 MLflow Run ID: 2aa45455ac674c838fd6d795698e8336
⚠️ SageMaker Model Registry registration skipped due to ECR access issues

🎉 PIPELINE COMPLETED SUCCESSFULLY!
🥇 Best Model: LogisticRegression
🎯 Best AUC Score: 0.5237
🎪 Accuracy: 0.7350
📦 MLflow Model Name: SalesConversion-LogisticRegression
📝 MLflow Run ID: 2aa45455ac674c838fd6d795698e8336
🔗 MLflow Tracking URI: arn:aws:sagemaker:ap-south-1:394376288194:mlflow-tracking-server/final-capstone-aravind-mlflow
🔥 ONLY THE BEST MODEL HAS BEEN REGISTERED IN MLFLOW MODEL REGISTRY
⚠️ SageMaker Model Registry registration disabled due to ECR access